In [129]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import sys
import os
import pickle
import librosa
import librosa.display
from IPython.display import Audio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [130]:
df = pd.read_csv("Data\\features_3_sec.csv")
df.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues


In [131]:
df = df.drop(labels='length',axis=1)

In [132]:
df = df.drop(labels='filename',axis=1)

In [133]:
class_list = df.iloc[:, -1]
convertor = LabelEncoder()

In [134]:
y = convertor.fit_transform(class_list)

In [135]:
from sklearn.preprocessing import StandardScaler
fit = StandardScaler()
X = fit.fit_transform(np.array(df.iloc[:, :-1], dtype = float))

In [136]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [138]:
def build_model():
    model = Sequential()
    model.add(keras.layers.Dense(1024, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(keras.layers.Dropout(0.2))

    model.add(keras.layers.Dense(512, activation='relu'))
    model.add(keras.layers.Dropout(0.2))

    model.add(keras.layers.Dense(256, activation='relu'))
    model.add(keras.layers.Dropout(0.2))

    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dropout(0.2))

    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.2))

    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(keras.layers.Dropout(0.2))

    model.add(keras.layers.Dense(10, activation='softmax'))
    
    return model 

In [139]:
model = build_model()
model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.001),
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

In [140]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 1024)              59392     
                                                                 
 dropout_16 (Dropout)        (None, 1024)              0         
                                                                 
 dense_20 (Dense)            (None, 512)               524800    
                                                                 
 dropout_17 (Dropout)        (None, 512)               0         
                                                                 
 dense_21 (Dense)            (None, 256)               131328    
                                                                 
 dropout_18 (Dropout)        (None, 256)               0         
                                                                 
 dense_22 (Dense)            (None, 128)              

In [141]:
%%timeit -n1 -r1
with tf.device('/GPU:0'):
    model.fit(X_train,y_train,epochs=60,validation_data=(X_test, y_test), batch_size=128)
    model.save("model_trained.h5")
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

Epoch 1/60
53/53 [==============================] - 1s 15ms/step - loss: 1.8749 - accuracy: 0.3181 - val_loss: 1.3577 - val_accuracy: 0.5475
Epoch 2/60
53/53 [==============================] - 0s 8ms/step - loss: 1.3688 - accuracy: 0.5256 - val_loss: 1.0312 - val_accuracy: 0.6463
Epoch 3/60
53/53 [==============================] - 0s 7ms/step - loss: 1.0965 - accuracy: 0.6287 - val_loss: 0.8645 - val_accuracy: 0.7179
Epoch 4/60
53/53 [==============================] - 0s 7ms/step - loss: 0.9288 - accuracy: 0.6900 - val_loss: 0.7342 - val_accuracy: 0.7595
Epoch 5/60
53/53 [==============================] - 0s 7ms/step - loss: 0.8036 - accuracy: 0.7508 - val_loss: 0.6487 - val_accuracy: 0.7944
Epoch 6/60
53/53 [==============================] - 0s 8ms/step - loss: 0.6982 - accuracy: 0.7825 - val_loss: 0.6073 - val_accuracy: 0.7998
Epoch 7/60
53/53 [==============================] - 0s 7ms/step - loss: 0.6223 - accuracy: 0.8011 - val_loss: 0.6235 - val_accuracy: 0.7992
Epoch 8/60
53/53 [=

In [81]:
import librosa
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
def extract_features(data, sr):
    # Extract Mel-frequency cepstral coefficients (MFCCs)
    mfccs = librosa.feature.mfcc(y=data, sr=sr, n_mfcc=20)
    return mfccs.flatten()
def extracttion(path):
    audio_file_path = path
    data, sr = librosa.load(audio_file_path)

    # Extract features (similar to your training data preprocessing)
    features = extract_features(data, sr)  # You need to define extract_features based on your preprocessing
    features = features[:58]
    # Reshape features to a 2D array (1 row, multiple columns)
    features_reshaped = features.reshape(1, -1)

    # Standardize the features using the same scaler used during training
    features_standardized = fit.transform(features_reshaped)  # Assuming 'fit' is your StandardScaler
    
    return features_standardized

features_x = extracttion("E:\code_files\\viki_dl\Data\genres_original\\disco\\disco.00056.wav")
model_trained  = load_model("E:\code_files\\viki_dl\\model_trained.h5")
prediction = model_trained.predict(features_x)

# Convert the prediction to a genre label
predicted_label = np.argmax(prediction)
predicted_genre = convertor.inverse_transform([predicted_label])[0]

# Display the predicted genre
print("Predicted Genre:", predicted_genre)

1/1 [==============================] - 0s 83ms/step
Predicted Genre: reggae


In [155]:
from glob import glob
def feature_extracter(path):
        num_segment=1
        num_mfcc=20
        sample_rate=22050
        n_fft=2048
        hop_length=512
        my_csv={"chroma_stft_mean": [], "chroma_stft_var": [], "rms_mean": [], "rms_var": [], "spectral_centroid_mean": [],
                "spectral_centroid_var": [], "spectral_bandwidth_mean": [], "spectral_bandwidth_var": [], "rolloff_mean": [], "rolloff_var": [],
                "zero_crossing_rate_mean": [], "zero_crossing_rate_var": [], "harmony_mean": [], "harmony_var": [], "perceptr_mean": [],
                "perceptr_var": [], "tempo": [], "mfcc1_mean": [], "mfcc1_var" : [], "mfcc2_mean" : [], "mfcc2_var" : [],
                "mfcc3_mean" : [], "mfcc3_var" : [], "mfcc4_mean" : [], "mfcc4_var" : [], "mfcc5_mean" : [], 
                "mfcc5_var" : [], "mfcc6_mean" : [], "mfcc6_var" : [], "mfcc7_mean" : [], "mfcc7_var" : [],
                "mfcc8_mean" : [], "mfcc8_var" : [], "mfcc9_mean" : [], "mfcc9_var" : [], "mfcc10_mean" : [], 
                "mfcc10_var" : [], "mfcc11_mean" : [], "mfcc11_var" : [], "mfcc12_mean" : [], "mfcc12_var" : [], 
                "mfcc13_mean" : [], "mfcc13_var" : [], "mfcc14_mean" : [], "mfcc14_var" : [], "mfcc15_mean" : [], 
                "mfcc15_var" : [], "mfcc16_mean" : [], "mfcc16_var" : [], "mfcc17_mean" : [], "mfcc17_var" : [], 
                "mfcc18_mean" : [], "mfcc18_var" : [], "mfcc19_mean" : [], "mfcc19_var" : [], "mfcc20_mean" : [], 
                "mfcc20_var":[]}
        my_3_csv=my_csv.copy()
        audio_file = path
        y, sr = librosa.load(audio_file, sr=sample_rate)
        chroma_hop_length = 512 #5000?
        chromagram = librosa.feature.chroma_stft(y=y, sr=sample_rate, hop_length=chroma_hop_length)
        my_csv["chroma_stft_mean"].append(chromagram.mean())
        my_csv["chroma_stft_var"].append(chromagram.var())

        #Root Mean Square Energy
        RMSEn= librosa.feature.rms(y=y)
        my_csv["rms_mean"].append(RMSEn.mean())
        my_csv["rms_var"].append(RMSEn.var())

        #Spectral Centroid
        spec_cent=librosa.feature.spectral_centroid(y=y)
        my_csv["spectral_centroid_mean"].append(spec_cent.mean())
        my_csv["spectral_centroid_var"].append(spec_cent.var())

        #Spectral Bandwith
        spec_band=librosa.feature.spectral_bandwidth(y=y,sr=sample_rate)
        my_csv["spectral_bandwidth_mean"].append(spec_band.mean())
        my_csv["spectral_bandwidth_var"].append(spec_band.var())

        #Rolloff
        spec_roll=librosa.feature.spectral_rolloff(y=y,sr=sample_rate)
        my_csv["rolloff_mean"].append(spec_roll.mean())
        my_csv["rolloff_var"].append(spec_roll.var())
        
        #Zero Crossing Rate
        zero_crossing=librosa.feature.zero_crossing_rate(y=y)
        my_csv["zero_crossing_rate_mean"].append(zero_crossing.mean())
        my_csv["zero_crossing_rate_var"].append(zero_crossing.var())

        #Harmonics and Perceptrual 
        harmony, perceptr = librosa.effects.hpss(y=y)
        my_csv["harmony_mean"].append(harmony.mean())
        my_csv["harmony_var"].append(harmony.var())
        my_csv["perceptr_mean"].append(perceptr.mean())
        my_csv["perceptr_var"].append(perceptr.var())

        #Tempo
        tempo, _ = librosa.beat.beat_track(y=y, sr = sr)
        my_csv["tempo"].append(tempo)
        mfcc=librosa.feature.mfcc(y=y,sr=sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
        mfcc=mfcc.T
        for x in range(20):
                feat1 = "mfcc" + str(x+1) + "_mean"
                feat2 = "mfcc" + str(x+1) + "_var"
                my_csv[feat1].append(mfcc[:,x].mean())
                my_csv[feat2].append(mfcc[:,x].var())
                
        df1 = pd.DataFrame(my_csv)
        feat = df1.values
        
        features_reshaped = feat.reshape(1, -1)

        features_standardized = fit.transform(features_reshaped) 
        return features_standardized
values = feature_extracter("E:\code_files\\viki_dl\Data\genres_original\disco\disco.00004.wav")
model_trained  = load_model("E:\code_files\\viki_dl\\viki_model.h5")
prediction = model_trained.predict(values)

# Convert the prediction to a genre label
predicted_label = np.argmax(prediction)
predicted_genre = convertor.inverse_transform([predicted_label])[0]

# Display the predicted genre
print("Predicted Genre:", predicted_genre)


1/1 [==============================] - 0s 375ms/step
Predicted Genre: disco
